# Multimodal RAG with Amazon Bedrock and Amazon Nova

This notebook demonstrates how to implement a multi-modal Retrieval-Augmented Generation (RAG) system using Amazon Bedrock with Amazon Nova and LangChain. Many documents contain a mixture of content types, including text and images. Traditional RAG applications often lose valuable information captured in images. With the emergence of Multimodal Large Language Models (MLLMs), we can now leverage both text and image data in our RAG systems.

In this notebook, we'll explore one approach to multi-modal RAG:

1. Use multimodal embeddings (such as Amazon Titan) to embed both images and text
2. Retrieve relevant information using similarity search
3. Pass raw images and text chunks to a multimodal LLM for answer synthesis using Amazon Nova

We'll use the following tools and technologies:

* [LangChain](https://python.langchain.com/v0.2/docs/introduction/) to build a multimodal RAG system
* [faiss](https://github.com/facebookresearch/faiss) for similarity search
* [Amazon Nova](https://docs.aws.amazon.com/nova/latest/userguide/what-is-nova.html ) for answer synthesis
* [Amazon Titan Multimodal Embeddings](https://docs.aws.amazon.com/bedrock/latest/userguide/titan-multiemb-models.html) for image embeddings
* [Amazon Bedrock](https://aws.amazon.com/bedrock/) for accessing powerful AI models, like the ones above
* [pymupdf](https://pymupdf.readthedocs.io/en/latest/) to parse images, text, and tables from documents (PDFs)
* [boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html) for interacting with Amazon Bedrock

For this demostration we'll use the pricing guide of a Standard Bank (a South African bank) transactional account. The fact that the pdf contain mostly tables make the data processing more cumbersome and the responses of the model sub-optimal.

While retrieval-augmented generation (RAG) performs well on standard text corpora, extending it
to tabular documents is not as straightforward. Unlike plain text, which follows a natural sequence
and benefits from a wide range of pretrained language embeddings, tables convey meaning through a two-dimensional layout, where each cell’s interpretation depends on both its row and
column context. Naive text extraction can often disrupt this structure, as it is unable
to identify layout-related cues such as merged cells, hierarchical headers, implicit headers, and other
spatial relationships. 

In [2]:
# Import libraries
import boto3
import faiss
import json

import pymupdf
import requests
import os
import logging
import numpy as np
import warnings
from tqdm import tqdm
from botocore.exceptions import ClientError
from langchain_text_splitters import RecursiveCharacterTextSplitter
from IPython import display
from functions import processing, model


logger = logging.getLogger(__name__)
logger.setLevel(logging.ERROR)

warnings.filterwarnings("ignore")

## Downloading data

In [3]:
url  = "https://www.standardbank.co.za/file_source/South%20Africa/PDF/Personal%20Pricing/2025/ACHIEVA_Bundled_Account_Pricing_Guide_2025.pdf"

In [4]:
# Set filename and path
filename = "ACHIEVA_Bundled_Account_Pricing_Guide_2025.pdf"
filepath = os.path.join("data", filename)

In [5]:

# Creat file directory if it doesn't exist
os.makedirs("data", exist_ok=True)

# Download the file
response = requests.get(url)
if response.status_code == 200:
    with open(filepath,'wb') as file:
        file.write(response.content)
    print(f"File downloaded succesfully: {filepath}")
else:
    print("Download failed. Staus code: {response.status_code}")

File downloaded succesfully: data/ACHIEVA_Bundled_Account_Pricing_Guide_2025.pdf


In [91]:
doc = pymupdf.open(filepath)
num_pages = len(doc)
base_dir = "data"

# Creating the directories

processing.create_directories(base_dir)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=700, chunk_overlap=200, length_function=len)
items = []

# Process each page of the Pdf

for page_num in tqdm(range(num_pages), desc = 'Processing pages'):
    page = doc[page_num]
    text = page.get_text().replace('\n',' ')
    processing.process_tables(filepath,doc,page_num, base_dir,items)
    processing.process_text_chunks(filepath,text, text_splitter, page_num, base_dir, items)
    processing.process_images(filepath,doc, page, page_num, base_dir, items)
    processing.process_page_images(page, page_num, base_dir, items)

Processing pages: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 14/14 [00:01<00:00,  7.35it/s]


In [92]:
# Looking at the first text item
[i for i in items if i['type'] == 'text'][0]

{'page': 0,
 'type': 'text',
 'text': '2025 pricing Your ACHIEVA TM Account',
 'path': 'data/text/ACHIEVA_Bundled_Account_Pricing_Guide_2025.pdf_text_0_0.txt'}

In [93]:
[i for i in items if i['type'] == 'table'][0]

{'page': 1,
 'type': 'table',
 'text': 'WWhhaatt  yyoouu  ggeett: nan, ',
 'path': 'data/tables/ACHIEVA_Bundled_Account_Pricing_Guide_2025.pdf_table_1_0.txt'}

## Generating Multimodal Embeddings

In [94]:
# Set embedding vector dimension
embedding_vector_dimension = 384

# Coun the number of each type of item

item_counts = {
    'text': sum(1 for item in items if item['type'] == 'text'),
    'table': sum(1 for item in items if item['type'] == 'table'),
    'image': sum(1 for item in items if item['type'] == 'image'),
    'page': sum(1 for item in items if item['type'] == 'page')
}    
    


In [95]:
item_counts


{'text': 33, 'table': 22, 'image': 9, 'page': 14}

In [96]:
# Initialize the counters
counters = dict.fromkeys(item_counts.keys(),0)

In [97]:
# Generate embeddings for all items
with tqdm(
    total=len(items),
    desc="Generating embeddings",
    bar_format=(
        "{l_bar}{bar}| {n_fmt}/{total_fmt} "
        "[{elapsed}<{remaining}, {rate_fmt}{postfix}]"
    )
) as pbar:
    
    for item in items:
        item_type = item['type']
        counters[item_type] += 1
        
        if item_type in ['text', 'table']:
            # For text or table, use the formatted text representation
            item['embedding'] = model.generate_multimodal_embeddings(prompt=item['text'],output_embedding_length=embedding_vector_dimension) 
        else:
            # For images, use the base64-encoded image data
            item['embedding'] = model.generate_multimodal_embeddings(image=item['image'], output_embedding_length=embedding_vector_dimension)
        
        # Update the progress bar
        pbar.set_postfix_str(f"Text: {counters['text']}/{item_counts['text']}, Table: {counters['table']}/{item_counts['table']}, Image: {counters['image']}/{item_counts['image']}")
        pbar.update(1)

Generating embeddings: 100%|█████████████████████████████████████████████████████████████████████| 78/78 [00:20<00:00,  3.82it/s, Text: 33/33, Table: 22/22, Image: 9/9]


## Create vector database/index

In [98]:
all_embeddings = np.array([item['embedding'] for item in items],dtype=np.float32)

# Create FAISS Index
index = faiss.IndexFlatL2(embedding_vector_dimension)

# Clear any pre-existing index
index.reset()

# Add embeddings to the index
index.add(all_embeddings)

## Test the RAG pipeline

In [99]:
def generate_response(query, output_embedding_length):

    query_embedding = model.generate_multimodal_embeddings(prompt=query,output_embedding_length=embedding_vector_dimension)

    # Search for the nearest neighbors in the vector database
    distances, result = index.search(np.array(query_embedding, dtype=np.float32).reshape(1,-1), k=5)
    # Retrieve the matched items
    matched_items = [{k: v for k, v in items[index].items() if k != 'embedding'} for index in result.flatten()]

    # Generate RAG response with Amazon Nova
    response = model.invoke_nova_multimodal(prompt=query, matched_items=matched_items)

    return response

In [100]:
query = 'What is the monthly administration fee of the ACHIEVA account?'

response = generate_response(query=query, output_embedding_length=embedding_vector_dimension)

In [101]:
# Display the response
display.Markdown(response)

The text context provided does not explicitly state the monthly administration fee for the ACHIEVA account. It mentions that the monthly fee will remain unchanged starting from 1 January 2025, but it does not specify the exact amount. 

However, it does mention additional fees for optional features:
- Personalised ACHIEVA Gold Cheque Card: R115 per month
- Personalised ACHIEVA Gold Credit Card: an extra R63 per month

If you need the exact monthly administration fee for the ACHIEVA account, you may need to contact Standard Bank directly or visit their official website for the most current and detailed information.

--------

The above response is not great and you can see that the model is struggling to find the correct response. If the information wasn't split accross columns (or tables), the model would have found it easier to extract the required information 

--------

In [102]:
query = 'List the ATM withdrawal fees for this account'

response = generate_response(query=query, output_embedding_length=embedding_vector_dimension)

# Display the response
display.Markdown(response)

Sure, here are the ATM withdrawal fees for this account:

1. **Standard Bank ATM Withdrawals:**
   - R2.65 per R100 or part thereof

2. **Other Bank ATM Withdrawals:**
   - R2.65 per R100 or part thereof

3. **International ATM Withdrawals:**
   - R3 per R100 or part thereof (min R70)
   - Plus an International transaction fee of 2.75%

4. **Coin Withdrawals:**
   - Not available at ATMs

5. **Notes and Coin Withdrawals:**
   - Not available at ATMs

6. **Cash for Cash (Change):**
   - Not available at ATMs

----

This is a better response!

----

In [103]:
query = 'What is included in this account?'

response = generate_response(query=query, output_embedding_length=embedding_vector_dimension)

# Display the response
display.Markdown(response)

Here's a breakdown of the information provided regarding various banking fees and services offered by Standard Bank:

### Deposit Fees
- **Notes via ATM:** R1.60 per R100 or part thereof.
- **Notes via Branch:** R90 + R4 per R100 or part thereof.
- **Coin Deposit via ATM:** Not available.
- **Coin Deposit via Branch:** R90 + R15 per R100 or part thereof.
- **Notes and Coin Deposit via ATM:** Not available.
- **Notes and Coin Deposit via Branch:** R90 + R4 per R100 (for notes) + R15 per R100 (for coins) or part thereof.

### Monthly Fees
- **Monthly Administration Fee:** R115.
- **Internet, Cellphone, and Banking App:** Free.

### Overdraft Fees
- **Monthly Service Fee:** R69 (applicable for both limited and non-limited accounts. For accounts with no overdraft limit, this fee is charged at month-end if the account is in debit balance by an amount of R200 or more).
- **Initiation Fee:** R74.75 + 11.5% of the limit. The maximum fee is R1,207.

### Statement Fees
- **Online Balance Enquiry:** Free.
- **ATM Balance Enquiry:** R1.
- **Branch Balance Enquiry:** R20.
- **Balance Enquiry (Other Bank):** R11.
- **Monthly Statements:** 
  - **Posted Statements:** R75 per statement.
  - **Free up to 6 months, thereafter R10 per month.**
- **eStatements:**
  - **Monthly:** R18.
  - **Weekly:** R35.
  - **Daily:** R60.

### Included Services
- **Self-service banking.**
- **Smart strategies.**

### Contact Information
- For more details, contact your branch.

### Security Notice
- Standard Bank will never ask for personal information over the phone or send links requiring you to capture your Internet Banking details. Stay safe and alert.

### Disclaimer
- Products, services, and terms may change. You will be informed of changes within a reasonable time. Read your contract carefully. For questions or more information, contact your branch.

### Regulatory Information
- Standard Bank subscribes to the Code

In [104]:
query = 'What is the features and perks of this account?'

response = generate_response(query=query, output_embedding_length=embedding_vector_dimension)

# Display the response
display.Markdown(response)

Certainly! Here are the features and perks of the account based on the provided text:

### Features and Perks:

#### **Deposits:**
- **Notes via ATM:** R1.60 per R100 or part thereof.
- **Notes via Branch:** R90 + R4 per R100 or part thereof.
- **Coin Deposit via Branch:** R90 + R15 per R100 or part thereof.
- **Notes and Coin Deposit via Branch:** R90 + R4 per R100 (for notes) + R15 per R100 (for coins) or part thereof.

#### **Monthly Fees:**
- **Monthly Administration Fee:** R115
- **Internet, Cellphone, and Banking App:** Free

#### **Self-Service Banking:**
- **In-app transaction notifications with MyUpdates.**

#### **Go Cashless & Cardless:**
- **Pay with your watch or fitness tracker:** Using Garmin Pay or Fitbit Pay.
- **Buy lotto tickets, prepaid airtime, or electricity:** On the Banking App or through Cellphone Banking by dialling *120*2345#.
- **Purchase value-added service vouchers:** Such as Spotify, Showmax, PlayStation, and Steam from the Banking App.

### Other Fees:
- **Pin Reset:**
  - **ATM:** Free
  - **Branch:** R15
- **Card Replacement:**
  - **Branch:** R160
- **Proof of Banking:**
  - **Online:** Free
  - **ATM:** R8.50
  - **Branch:** 1 Free per month, thereafter R45
- **Subsidy Letter:**
  - **Branch:** R22

### Additional Perks:
- **Smart Strategies:** Though not detailed, this suggests tools or advice for better financial management.
- **Contact Us:** Availability of customer support.
- **Transaction Fees:** Though not detailed, this suggests a fee schedule for various transactions.
- **Set Yourself Up for Success:** Likely refers to financial planning tools or resources.
- **Securing Your Future:** Probably refers to long-term financial planning or investment options.

----

Again the model is struggling to extract the relevant information from the document

---